In [ ]:
import numpy as np
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_dual_ur5e_and_table_to_builder
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.drake.visualization import publish_dual_arm_joint_path, publish_ik_solutions, add_meshcat_triad

In [ ]:
tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

In [ ]:
# Creating the default scene
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_indices, gripper_indices = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)
diagram, context = finish_build(robot_diagram_builder, meshcat)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[*arm_indices, *gripper_indices],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

is_state_valid_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
home_joints_left = np.deg2rad([180, -135, 95, -50, -90, -90])
home_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

arm_left_index, arm_right_index = arm_indices
plant.SetPositions(plant_context, arm_left_index, home_joints_left)
plant.SetPositions(plant_context, arm_right_index, home_joints_right)
diagram.ForcedPublish(context)

In [ ]:
transform_0 = RigidTransform(p=[0, 0, 0.35], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform_0.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame left", X_W_Triad=transform_0)

In [ ]:
transform_1 = RigidTransform(p=[0.15, 0, 0.3], rpy=RollPitchYaw([np.pi / 2, 0, np.pi / 2]))
tcp_pose_1 = np.ascontiguousarray(transform_1.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame right", X_W_Triad=transform_1)

In [ ]:
from typing import List
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from ur_analytic_ik import ur5e

y_distance = 0.45
X_W_L = RigidTransform(rpy=RollPitchYaw([0, 0, -np.pi / 2]), p=[0, -y_distance, 0]).GetAsMatrix4()
X_W_R = RigidTransform(rpy=RollPitchYaw([0, 0, -np.pi / 2]), p=[0, y_distance, 0]).GetAsMatrix4()

X_CB_B = RigidTransform(
    rpy=RollPitchYaw([0, 0, np.pi]), p=[0, 0, 0]
).GetAsMatrix4()  # 180 rotation between URDF base en control box base
X_LCB_W = X_CB_B @ np.linalg.inv(X_W_L)
X_RCB_W = X_CB_B @ np.linalg.inv(X_W_R)


def left_inverse_kinematics_fn(tcp_pose: HomogeneousMatrixType) -> List[JointConfigurationType]:
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(X_LCB_W @ tcp_pose, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions


def right_inverse_kinematics_fn(tcp_pose: HomogeneousMatrixType) -> List[JointConfigurationType]:
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(X_RCB_W @ tcp_pose, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions

In [ ]:
solutions_left = left_inverse_kinematics_fn(tcp_pose_0)
publish_ik_solutions(solutions_left, 2.0, meshcat, diagram, context, arm_left_index)

In [ ]:
solutions_right = right_inverse_kinematics_fn(tcp_pose_1)
publish_ik_solutions(solutions_right, 2.0, meshcat, diagram, context, arm_right_index)

In [ ]:
planner = DualArmOmplPlanner(
    is_state_valid_fn,
    left_inverse_kinematics_fn,
    right_inverse_kinematics_fn,
)

In [ ]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, tcp_pose_0, None)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, None, tcp_pose_1)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)

In [ ]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, tcp_pose_0, tcp_pose_1)
publish_dual_arm_joint_path(path, 2.0, meshcat, diagram, context, *arm_indices)